## Installing YOLOv4/Darknet
=============================================

### Step 1: Install the Prerequisite

1. **CMake:** Download and Install Cmake from the Link. [[https://cmake.org/download/](https://cmake.org/download/)]
2. **CUDA:** Download and Install CUDA from the Link. [[https://developer.nvidia.com/cuda-downloads](https://developer.nvidia.com/cuda-downloads)]
3. **cuDNN:** Download and Install cuDNN from the Link. [[https://developer.nvidia.com/cudnn-download-survey](https://developer.nvidia.com/cudnn-download-survey)]
4. **OpenCV:** Run `pip install opencv-python` and Install OpenCV
5. **Visual Studio Community Edition:** Download Visual Studio from the Link and Install [[Microsoft Visual Studio Community Edition]](https://visualstudio.microsoft.com/thank-you-downloading-visual-studio/?sku=Community&rel=16)
    - While installing, Make Sure you choose **_Desktop Development with C++_** under the **Workloads** tab
6. **Powershell Command:** Open Powershell and execute the following command `Set-ExecutionPolicy unrestricted -Scope CurrentUser -Force`

### Step 2: Preparation for Installing Darknet

1. Open `install_darknet.py` in your favorite editor.
2. Find `VIRTUAL_ENV` and replace the value depending on your environment.
    - If you are using a Virtual Environment, set the **name of the environment** as the value of `VIRTUAL_ENV`
    - If you are not using any virtual environment, left the value of `VIRTUAL_ENV` blank

### Step 3: Installing Darknet

Go into the project directory and open powershell in the current folder. Now run `python install_darknet.py` to start installing darknet. As the installation may take a long time (1-2 hours), please don't panic and stop the execution of the script.

### Step 4: Removing the Garbage Files

After successfully installing the Darknet, Please check the followings:

1. Your project directory should contain `darknet` folders inside.
2. Start powershell in the project directory and run `python` to start interactive shell. Then try to import darknet using `import darknet`. If you don't see any error, Your installation is successful.

After successful running of the script, you will see a Windows explorer pop up from where delete the **darknet** folder.

**Congratulations, You can now use Darknet in your Projects**